## Config

In [1]:
### Configuration
# Dependencies
import os
import pandas as pd
import numpy as np
from summerplaylist import *

# Environment variables
CLIENTID = os.environ.get('CLIENTID')
CLIENTSECRET = os.environ.get('CLIENTSECRET')
REDIRECT_URI = 'https://example.com/callback'

# Filepaths
artistsKnownPath = os.path.join('data', 'artistsKnown.csv')

# Set max age for new artist songs
maxAge = 5

## Log In

In [4]:
# Authenticate to Spotify
credentials = authenticateToSpotify(CLIENTID, CLIENTSECRET, REDIRECT_URI)

Opening browser for Spotify login...


## Collect new artists

In [29]:
# # This block can be commented out once it has been run once. It does not update each year.

# # Collect names of new artists from Music Festival Wizard
# artistsNew = fetchMusicFestivalWizard()
# # Filter out known artists who we have already listened to
# artistsNew = artistsNew.loc[~artistsNew['artistName'].isin(artistsKnown['artistName'])]
# artistsNew = artistsNew.head()
# # Search spotify for artistIDs
# artistsNew['artistID'] = artistsNew.apply(findArtistIDs, credentials=credentials, axis=1)
# # Filter out if artistID was known
# artistsNew = artistsNew.loc[~artistsNew['artistID'].isin(artistsKnown['artistID'])]
# artistsNew

In [ ]:
# Website with a list of festivals
# https://www.thrillist.com/news/nation/music-festival-lineups-tickets-us-guide
# Coachella
# Governor's Ball
# Bonnaroo
# BottleRock
# Lollapalooza
# Boston Calling
# Outside Lands
# Pitchfork
# Shaky Knees



# Forecastle
# Panorama
# Newport Folk Festival
# Newport Jazz Festival
# Afropunk
# FYF
# Sloss

# Hangout
# Electric Forest
# Sasquatch

In [16]:
credentials[2]

RefreshingToken(access_token='BQB2v778N397oqqYn8mtD0D_i9QSf5Ha1bIgii7XNCMid9Cl30ORCwp0xJ_eIIVTKLbiGJwub9HMq8SppNJbtSGY6feiRBbPZbcRNCmVkMkPR84heZ9CG6yCg0dzQc4rFd-2w7sMkLh0B_ImCeEMsmJVSYWdHhqh4813dq5w9qu4NVXone1o-bbZ-u8K8geFHrtc5z8h2Me7zxnOQmKiDmYrYjUhSNzcvx7zQBddZyGynvWvhrSCx1c3VtqP9cT1bqbVD_uHHkdR2zpEeutP8qUGFH67ZMuxGZWz8WbfLryD4x0', refresh_token='AQDc1il6PxHTuMifWlBXOtuwVYP9a_a_j3oNqV1aZ5a84-8Fy80zm6-sgvxUvbYBqClXJjqJ4XkmJOROKbsU2whV_jSaK_ipvLKtOPpZBJuR2f0AKIlgMsAC0WAZqzJTjCs', scope=Scope('playlist-modify-private', 'playlist-modify-public', 'playlist-read-private', 'ugc-image-upload', 'user-follow-modify', 'user-library-modify', 'user-modify-playback-state'))

In [15]:
with credentials[0].token_as(credentials[2]):
    credentials[0].playlists(credentials[1])

TypeError: 'NoneType' object is not iterable

In [5]:
# Print the playlist IDs so user can select an id to add to "New Artists to Explore"
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta
from collections import Counter
import pandas as pd
import numpy as np
from requests_html import HTMLSession
import tekore as tk

spotify, userID, user_token = credentials
with spotify.token_as(user_token):
    # Check if playlist already exists; if not create a new one
    userPlaylists = pd.DataFrame([{'id':x.id, 'name':x.name} for x in spotify.playlists(userID, limit=40).items])

userPlaylists

TypeError: 'NoneType' object is not iterable

In [10]:
try:
    spotify.playlists(userID)
except tk.HTTPError as e:
    print(e)


TypeError: 'NoneType' object is not iterable

## Process new artists

In [30]:
# Read in known artists
artistsKnown = pd.read_csv(artistsKnownPath)

In [33]:
# Collect artists from an existing favorited playlist
playlistToFetch = '5DtWXe9QB55ATwZ6paG7Yl' #getPlaylistID(credentials, 'New Artists to Explore')
artistsNew = fetchTracksFromPlaylist(credentials, playlistToFetch)
artistsNew = artistsNew[['artistName', 'artistID']].drop_duplicates('artistID', ignore_index=True).reset_index(drop=True)
# Filter out if artistID was known
artistsNew = artistsNew.loc[~artistsNew['artistID'].isin(artistsKnown['artistID'])].drop_duplicates('artistID', ignore_index=True)
artistsNew

,artistName,artistID


## Get top songs

In [27]:
# Get their top three songs in last 5 years
songURIs = fetchTopSongs(credentials, artistsNew['artistID'], maxAge=maxAge)
# How many songs are in this round?
print(f"There are {len(songURIs)} songs about to be added")

There are 0 songs about to be added


## Put songs in playlist

In [28]:
# Warning: This will make a new playlist if it does not already exist
playlistID = getPlaylistID(credentials, 'Summer01 - Brand New Artists')
# Add songs to the playlist
addToPlaylist(credentials, songURIs, playlistID)

## Update the list of known artists

In [ ]:
# Append artistsNew to artistsKnown.csv 
artistsKnown = pd.concat([artistsKnown, artistsNew], axis=0, ignore_index=True)
artistsKnown.to_csv(artistsKnownPath, index=False)

## Create playlist to drop good songs into

In [10]:
# Cull exploreArtists into ArtistHasPotential playlist
winnersPlaylist = getPlaylistID(credentials, 'Summer02 - New Artists to Explore Further')
print(f'Now drop songs you like into this playlist: {winnersPlaylist}')

Now drop songs you like into this playlist: 6XiutFvGeKpupJ9BODpNoy
